In [1]:
import os
os.environ["TRAME_DISABLE_V3_WARNING"] = "1"
os.environ["TRAME_IFRAME_BUILDER"] = "jupyter-extension"
os.environ["TRAME_BACKEND"] = "jupyter"

In [2]:
from trame.app import get_server
from trame.decorators import TrameApp, change
from trame.widgets import vuetify, vtk as vtk_widgets
from trame.ui.vuetify import SinglePageLayout

from vtkmodules.vtkFiltersSources import vtkConeSource
from vtkmodules.vtkRenderingCore import (
    vtkRenderer,
    vtkRenderWindow,
    vtkRenderWindowInteractor,
    vtkPolyDataMapper,
    vtkActor,
)

# VTK factory initialization
from vtkmodules.vtkInteractionStyle import vtkInteractorStyleSwitch  # noqa
import vtkmodules.vtkRenderingOpenGL2  # noqa

@TrameApp()
class VTKCone:
    def __init__(self, server_name=None):
        self.server = get_server(server_name, client_type="vue2")

        # VTK
        renderer = vtkRenderer()
        renderWindow = vtkRenderWindow()
        renderWindow.AddRenderer(renderer)
        renderWindow.OffScreenRenderingOn()

        renderWindowInteractor = vtkRenderWindowInteractor()
        renderWindowInteractor.SetRenderWindow(renderWindow)
        renderWindowInteractor.GetInteractorStyle().SetCurrentStyleToTrackballCamera()

        cone_source = vtkConeSource()
        mapper = vtkPolyDataMapper()
        actor = vtkActor()
        
        mapper.SetInputConnection(cone_source.GetOutputPort())
        actor.SetMapper(mapper)
        renderer.AddActor(actor)
        renderer.ResetCamera()

        self.source = cone_source
        self.render_window = renderWindow
        
        # UI
        self.ui = self._generate_ui()

    @change("resolution")
    def _on_resolution_change(self, resolution, **kwargs):
        self.source.SetResolution(resolution)
        self.ctrl.view_update()

    @property
    def ctrl(self):
        return self.server.controller

    @property
    def state(self):
        return self.server.state

    @property
    def resolution(self):
        return self.state.resolution

    @resolution.setter
    def resolution(self, v):
        with self.state:
            self.state.resolution = v

    def reset_resolution(self):
        self.resolution = 6

    def _generate_ui(self):
        with SinglePageLayout(self.server) as layout:
            layout.title.set_text("VTK Rendering")
            with layout.toolbar as toolbar:
                toolbar.dense = True
                vuetify.VSpacer()
                vuetify.VSlider(
                    v_model=("resolution", 6),
                    min=3,
                    max=60,
                    step=1,
                    hide_details=True,
                    style="max-width: 300px;",
                )
                with vuetify.VBtn(icon=True, click=self.reset_resolution):
                    vuetify.VIcon("mdi-lock-reset")
                with vuetify.VBtn(icon=True, click=self.ctrl.view_reset_camera):
                    vuetify.VIcon("mdi-crop-free")

            with layout.content:
                with vuetify.VContainer(fluid=True, classes="pa-0 fill-height"):
                    with vtk_widgets.VtkRemoteView(self.render_window) as view:
                        self.ctrl.view_reset_camera = view.reset_camera
                        self.ctrl.view_update = view.update

            return layout


In [3]:
c2 = VTKCone("vtk")
await c2.ui.ready
c2.ui

In [4]:
c2.ui